# **Hi there** ,

This Notebook will deal with an implementation of 
5 Layered CNN architecture
* **Resnet**
* **VGG16**
* **VGG19**
* **Imagnet**
* **Inception Model**

In [21]:
import numpy as np
import pickle
import cv2
from os import listdir
from sklearn.preprocessing import LabelBinarizer
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation, Flatten, Dropout, Dense
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.preprocessing import image
from keras.preprocessing.image import img_to_array
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [22]:
import os
os.listdir("/kaggle/input/plantvillage-dataset/color")

['Tomato___Late_blight',
 'Tomato___healthy',
 'Grape___healthy',
 'Orange___Haunglongbing_(Citrus_greening)',
 'Soybean___healthy',
 'Squash___Powdery_mildew',
 'Potato___healthy',
 'Corn_(maize)___Northern_Leaf_Blight',
 'Tomato___Early_blight',
 'Tomato___Septoria_leaf_spot',
 'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot',
 'Strawberry___Leaf_scorch',
 'Peach___healthy',
 'Apple___Apple_scab',
 'Tomato___Tomato_Yellow_Leaf_Curl_Virus',
 'Tomato___Bacterial_spot',
 'Apple___Black_rot',
 'Blueberry___healthy',
 'Cherry_(including_sour)___Powdery_mildew',
 'Peach___Bacterial_spot',
 'Apple___Cedar_apple_rust',
 'Tomato___Target_Spot',
 'Pepper,_bell___healthy',
 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)',
 'Potato___Late_blight',
 'Tomato___Tomato_mosaic_virus',
 'Strawberry___healthy',
 'Apple___healthy',
 'Grape___Black_rot',
 'Potato___Early_blight',
 'Cherry_(including_sour)___healthy',
 'Corn_(maize)___Common_rust_',
 'Grape___Esca_(Black_Measles)',
 'Raspberry___healthy'

In [ ]:
EPOCHS = 5
INIT_LR = 1e-3
BS = 32
default_image_size = tuple((224, 224))
image_size = 0
directory_root = '/kaggle/input/plantvillage-dataset/'
width=224
height=224
depth=3

In [52]:
def convert_image_to_array(image_dir):
    try:
        image = cv2.imread(image_dir)
        if image is not None :
            image = cv2.resize(image, default_image_size)   
            return img_to_array(image)
        else :
            return np.array([])
    except Exception as e:
        print(f"Error : {e}")
        return None

In [ ]:
image_list, label_list, classes = [], [], []
try:
    print("[INFO] Loading images ...")
    root_dir = listdir(directory_root)
    for directory in root_dir :
        # remove .DS_Store from list
        if directory == ".DS_Store" :
            root_dir.remove(directory)

    for plant_folder in ['color/'] :
        print(plant_folder)
        plant_disease_folder_list = listdir(f"{directory_root}/{plant_folder}")
        print(plant_disease_folder_list)
        for disease_folder in plant_disease_folder_list :
            # remove .DS_Store from list
            if disease_folder == ".DS_Store" :
                plant_disease_folder_list.remove(disease_folder)

        for plant_disease_folder in plant_disease_folder_list:
            print(f"[INFO] Processing {plant_disease_folder} ...")
            plant_disease_image_list = listdir(f"{directory_root}/{plant_folder}/{plant_disease_folder}")
            for single_plant_disease_image in plant_disease_image_list :
                if single_plant_disease_image == ".DS_Store" :
                    plant_disease_image_list.remove(single_plant_disease_image)

            for image in plant_disease_image_list:
                image_directory = f"{directory_root}/{plant_folder}/{plant_disease_folder}/{image}"
                if image_directory.endswith(".jpg") == True or image_directory.endswith(".JPG") == True:
                    image_list.append(convert_image_to_array(image_directory))
                    label_list.append(plant_disease_folder)
            classes.append(plant_disease_folder)
    print("[INFO] Image loading completed")  
except Exception as e:
    print(f"Error --- : {e}")

In [25]:
len(classes)    

38

In [37]:
classes

['Tomato___Late_blight',
 'Tomato___healthy',
 'Grape___healthy',
 'Orange___Haunglongbing_(Citrus_greening)',
 'Soybean___healthy',
 'Squash___Powdery_mildew',
 'Potato___healthy',
 'Corn_(maize)___Northern_Leaf_Blight',
 'Tomato___Early_blight',
 'Tomato___Septoria_leaf_spot',
 'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot',
 'Strawberry___Leaf_scorch',
 'Peach___healthy',
 'Apple___Apple_scab',
 'Tomato___Tomato_Yellow_Leaf_Curl_Virus',
 'Tomato___Bacterial_spot',
 'Apple___Black_rot',
 'Blueberry___healthy',
 'Cherry_(including_sour)___Powdery_mildew',
 'Peach___Bacterial_spot',
 'Apple___Cedar_apple_rust',
 'Tomato___Target_Spot',
 'Pepper,_bell___healthy',
 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)',
 'Potato___Late_blight',
 'Tomato___Tomato_mosaic_virus',
 'Strawberry___healthy',
 'Apple___healthy',
 'Grape___Black_rot',
 'Potato___Early_blight',
 'Cherry_(including_sour)___healthy',
 'Corn_(maize)___Common_rust_',
 'Grape___Esca_(Black_Measles)',
 'Raspberry___healthy'

In [26]:
image_size = len(image_list)
image_size

54303

In [27]:
label_binarizer = LabelBinarizer()
image_labels = label_binarizer.fit_transform(label_list)
pickle.dump(label_binarizer,open('label_transform.pkl', 'wb'))
n_classes = len(label_binarizer.classes_)
print(label_binarizer.classes_)

['Apple___Apple_scab' 'Apple___Black_rot' 'Apple___Cedar_apple_rust'
 'Apple___healthy' 'Blueberry___healthy'
 'Cherry_(including_sour)___Powdery_mildew'
 'Cherry_(including_sour)___healthy'
 'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot'
 'Corn_(maize)___Common_rust_' 'Corn_(maize)___Northern_Leaf_Blight'
 'Corn_(maize)___healthy' 'Grape___Black_rot'
 'Grape___Esca_(Black_Measles)'
 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)' 'Grape___healthy'
 'Orange___Haunglongbing_(Citrus_greening)' 'Peach___Bacterial_spot'
 'Peach___healthy' 'Pepper,_bell___Bacterial_spot'
 'Pepper,_bell___healthy' 'Potato___Early_blight' 'Potato___Late_blight'
 'Potato___healthy' 'Raspberry___healthy' 'Soybean___healthy'
 'Squash___Powdery_mildew' 'Strawberry___Leaf_scorch'
 'Strawberry___healthy' 'Tomato___Bacterial_spot' 'Tomato___Early_blight'
 'Tomato___Late_blight' 'Tomato___Leaf_Mold' 'Tomato___Septoria_leaf_spot'
 'Tomato___Spider_mites Two-spotted_spider_mite' 'Tomato___Target_Spot'
 'Tomato___Tom

# **Normalization Of images**

In [28]:
np_image_list = np.array(image_list, dtype=np.float16) / 225.0
print("[INFO] Spliting data to train, test")
x_train, x_test, y_train, y_test = train_test_split(np_image_list, image_labels, test_size=0.2, random_state = 42) 

[INFO] Spliting data to train, test


In [29]:
y_train[0].shape

(38,)

In [30]:
x_train[0].shape

(128, 128, 3)

# **Augmentation Of data**

In [31]:
aug = ImageDataGenerator(
    rotation_range=25, width_shift_range=0.1,
    height_shift_range=0.1, shear_range=0.2, 
    zoom_range=0.2,horizontal_flip=True, 
    fill_mode="nearest")

# ***5 Layer CNN Algorithm ***

In [32]:
model = Sequential()
inputShape = (height, width, depth)
chanDim = -1
if K.image_data_format() == "channels_first":
    inputShape = (depth, height, width)
    chanDim = 1
model.add(Conv2D(32, (3, 3), padding="same",input_shape=inputShape))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(MaxPooling2D(pool_size=(3, 3)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(Conv2D(64, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(128, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(Conv2D(128, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(1024))
model.add(Activation("relu"))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(n_classes))
model.add(Activation("softmax"))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 128, 128, 32)      896       
_________________________________________________________________
activation_1 (Activation)    (None, 128, 128, 32)      0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 128, 128, 32)      128       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 42, 42, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 42, 42, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 42, 42, 64)        18496     
_________________________________________________________________
activation_2 (Activation)    (None, 42, 42, 64)       

# **Optimizer**

In [33]:
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
# distribution
model.compile(loss="binary_crossentropy", optimizer=opt,metrics=["accuracy"])
# train the network
print("[INFO] training network...")

[INFO] training network...


In [ ]:
history = model.fit_generator(
    aug.flow(x_train, y_train, batch_size=BS),
    validation_data=(x_test, y_test),
    steps_per_epoch=len(x_train) // BS,
    epochs=5, verbose=1
    )

In [48]:
import tensorflow as tf
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras import layers, Model

# Define the input layer
input_layer = tf.keras.Input(shape=(224, 224, 3), name='input_2')

# Define the DenseNet121 model with pretrained ImageNet weights
densenet_model = DenseNet121(weights='imagenet', include_top=False)

# Set the trainable parameter to False
densenet_model.trainable = False

# Pass the input through the DenseNet121 model
densenet_output = densenet_model(input_layer)

# Add GlobalAveragePooling2D layer
global_avg_pooling = layers.GlobalAveragePooling2D()(densenet_output)

# Add Dense layer for classification
output_layer = layers.Dense(38, activation='softmax', name='dense')(global_avg_pooling)

# Create the model
model = Model(inputs=input_layer, outputs=output_layer)

# Compile the model (if necessary)
# model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Print the summary of the model
model.summary()



Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
densenet121 (Model)          multiple                  7037504   
_________________________________________________________________
global_average_pooling2d_1 ( (None, 1024)              0         
_________________________________________________________________
dense (Dense)                (None, 38)                38950     
Total params: 7,076,454
Trainable params: 38,950
Non-trainable params: 7,037,504
_________________________________________________________________


In [49]:
model.load_weights('/kaggle/input/densenet121-pdd-model-v1/densenet121pdd_model.h5')

In [35]:
#model.save('model5layercnnpdd.h5')

In [62]:
image = convert_image_to_array('/kaggle/input/plantvillage-dataset/color/Tomato___healthy/000146ff-92a4-4db6-90ad-8fce2ae4fddd___GH_HL Leaf 259.1.JPG')
image = np.expand_dims(image, axis = 0)
print(image.shape)

(1, 224, 224, 3)


In [63]:
prediction = model.predict(image)

In [64]:
prediction

array([[5.9298141e-05, 5.0526167e-05, 7.3088362e-05, 8.6295448e-04,
        1.4432958e-08, 1.8327624e-05, 8.5813506e-03, 3.2632956e-06,
        4.2101906e-06, 2.2527824e-05, 5.6152180e-04, 3.3428162e-07,
        1.4459977e-06, 7.0532442e-06, 4.4071578e-04, 2.9661587e-05,
        2.9568456e-04, 8.0570629e-08, 1.5194784e-07, 8.7769717e-05,
        2.1700932e-04, 1.5882117e-04, 5.5384706e-05, 9.3481259e-07,
        5.0698850e-02, 5.5012974e-04, 1.0957169e-08, 2.1957824e-06,
        9.5698156e-04, 2.7584008e-05, 1.1188285e-04, 2.9185131e-07,
        2.8737481e-05, 3.2867814e-05, 6.1563128e-03, 1.8348687e-06,
        1.8194839e-06, 9.2989832e-01]], dtype=float32)

In [67]:
prediction.argmax()

37

In [68]:
classes[37]

IndexError: list index out of range

In [61]:
model.predict

<bound method Model.predict of <tensorflow.python.keras.engine.training.Model object at 0x7f4e6e915d68>>

# **Results and Plots**

In [ ]:
'''def Res_Plot_Test_Save(name):
    print("[INFO] Plotting model accuracy and Loss")
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs = range(1, len(acc) + 1)
    #Train and validation accuracy
    plt.plot(epochs, acc, 'b', label='Training accurarcy')
    plt.plot(epochs, val_acc, 'r', label='Validation accurarcy')
    plt.title('Training and Validation accurarcy')
    plt.legend()

    plt.figure()
    #Train and validation loss
    plt.plot(epochs, loss, 'b', label='Training loss')
    plt.plot(epochs, val_loss, 'r', label='Validation loss')
    plt.title('Training and Validation loss')
    plt.legend()
    plt.show()
    print("[INFO] Saving model")
    model.save(name)
    '''
print("[INFO] Calculating model accuracy")
scores = model.evaluate(x_test, y_test)
print(f"Test Accuracy: {scores[1]*100}")
    
    
    

In [ ]:
Res_Plot_Test_Save('model5layercnnpdd1.h5')

# Transfer Learning Implementation

### Transfer Learning Advantages
Typically transfer learning enables us to build more robust models which can perform a wide variety of tasks.
* Helps solve complex real-world problems with several constraints
* Tackle problems like having little or almost no labeled data availability
* Ease of transfering knowledge from one model to another based on domains and tasks
* Provides a path towards achieving Artificial General Intelligence some day in the future!

**For References Check out**
[https://towardsdatascience.com/a-comprehensive-hands-on-guide-to-transfer-learning-with-real-world-applications-in-deep-learning-212bf3b2f27a]

### **Inception ModelV3**

##### Weights : Imagenet

In [ ]:
from keras.models import Model
from keras.optimizers import Adam
from keras.layers import GlobalAveragePooling2D
from keras.layers import Dense
from keras.applications.inception_v3 import InceptionV3
from keras.utils.np_utils import to_categorical

# Get the InceptionV3 model so we can do transfer learning
base_inception = InceptionV3(weights='imagenet', include_top=False, 
                             input_shape=(128, 128, 3))
                             
# Add a global spatial average pooling layer
out = base_inception.output
out = GlobalAveragePooling2D()(out)
out = Dense(512, activation='relu')(out)
out = Dense(512, activation='relu')(out)
predictions = Dense(n_classes, activation='softmax')(out)
model = Model(inputs=base_inception.input, outputs=predictions)

# only if we want to freeze layers
for layer in base_inception.layers:
    layer.trainable = False
    
# Compile 
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss="binary_crossentropy", optimizer=opt,metrics=["accuracy"]) 
model.summary()

In [ ]:
history = model.fit_generator(
    aug.flow(x_train, y_train, batch_size=BS),
    validation_data=(x_test, y_test),
    steps_per_epoch=len(x_train) // BS,
    epochs=EPOCHS, verbose=1
    )

In [ ]:
model.save('inceptionV3_modelpdd_v3_1.h5')

In [ ]:
Res_Plot_Test_Save('Inception_ModelV3_(Imagenet_Weights)_v4.h5')

### **Resnet Model**

##### Weights : Imagenet

In [ ]:
from keras.models import Model
from keras import applications
from keras.optimizers import Adam
from keras.layers import GlobalAveragePooling2D
from keras.layers import Dense
from keras.applications.inception_v3 import InceptionV3
from keras.utils.np_utils import to_categorical

# Get the Resnet model so we can do transfer learning
base_model = applications.resnet50.ResNet50(weights= None, include_top=False, input_shape= (128,128,3))
                             
# Add a global spatial average pooling layer
out = base_model.output
out = GlobalAveragePooling2D()(out)
out = Dense(512, activation='relu')(out)
out = Dense(512, activation='relu')(out)
predictions = Dense(n_classes, activation='softmax')(out)
model = Model(inputs=base_model.input, outputs=predictions)

# only if we want to freeze layers
for layer in base_inception.layers:
    layer.trainable = False
    
# Compile 
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss="binary_crossentropy", optimizer=opt,metrics=["accuracy"]) 
model.summary()

In [ ]:
history = model.fit_generator(
    aug.flow(x_train, y_train, batch_size=BS),
    validation_data=(x_test, y_test),
    steps_per_epoch=len(x_train) // BS,
    epochs=5, verbose=1
    )

In [ ]:
model.save('resnet50_pdd_model.h5')

In [ ]:
Res_Plot_Test_Save('Resnet_With_Imagenet_Weights_model.h5')

### **VGG16 Model**

##### Weights : Imagenet

In [ ]:
from keras.models import Model
from keras.applications import vgg16
from keras.optimizers import Adam
from keras.layers import GlobalAveragePooling2D
from keras.layers import Dense
from keras.applications.inception_v3 import InceptionV3
from keras.utils.np_utils import to_categorical

# Get the Resnet model so we can do transfer learning
vgg = vgg16.VGG16(weights= None, include_top=False, input_shape= (128,128,3))
                             
# Add a global spatial average pooling layer
out = base_model.output
out = GlobalAveragePooling2D()(out)
out = Dense(512, activation='relu')(out)
out = Dense(512, activation='relu')(out)
predictions = Dense(n_classes, activation='softmax')(out)
model = Model(inputs=base_model.input, outputs=predictions)

# only if we want to freeze layers
for layer in base_inception.layers:
    layer.trainable = False
    
# Compile 
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss="binary_crossentropy", optimizer=opt,metrics=["accuracy"]) 
model.summary()

In [ ]:
history = model.fit_generator(
    aug.flow(x_train, y_train, batch_size=BS),
    validation_data=(x_test, y_test),
    steps_per_epoch=len(x_train) // BS,
    epochs=5, verbose=1
    )

In [ ]:
model.save('vgg16_model_pdd1.h5')

In [ ]:
Res_Plot_Test_Save('VGG16_With_Imagenet_Weingts.h5')

### **VGG19 Model**

##### Weights : Imagenet

In [ ]:
from keras.models import Model
from keras.applications import vgg19
from keras.optimizers import Adam
from keras.layers import GlobalAveragePooling2D
from keras.layers import Dense
from keras.applications.inception_v3 import InceptionV3
from keras.utils.np_utils import to_categorical

# Get the Resnet model so we can do transfer learning
vgg = vgg19.VGG19(weights= None, include_top=False, input_shape= (128,128,3))
                             
# Add a global spatial average pooling layer
out = base_model.output
out = GlobalAveragePooling2D()(out)
out = Dense(512, activation='relu')(out)
out = Dense(512, activation='relu')(out)
predictions = Dense(n_classes, activation='softmax')(out)
model = Model(inputs=base_model.input, outputs=predictions)

# only if we want to freeze layers
for layer in base_inception.layers:
    layer.trainable = False
    
# Compile 
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss="binary_crossentropy", optimizer=opt,metrics=["accuracy"]) 
model.summary()

In [ ]:
history = model.fit_generator(
    aug.flow(x_train, y_train, batch_size=BS),
    validation_data=(x_test, y_test),
    steps_per_epoch=len(x_train) // BS,
    epochs=15, verbose=1
    )

In [ ]:
model.save('vgg19_modelpdd_2.h5')

In [ ]:
Res_Plot_Test_Save('VGG19_Imagenet.h5')

# Transfer Learning Challenges
Transfer learning has immense potential and is a commonly required enhancement for existing learning algorithms. Yet, there are certain pertinent issues related to transfer learning that need more research and exploration. Apart from the difficulty of answering the questions of what, when, and how to transfer, negative transfer and transfer bounds present major challenges.
* Negative Transfer: The cases we have discussed so far talk about improvements in target tasks based on knowledge transfer from the source task. There are cases when transfer learning can lead to a drop in performance. Negative transfer refers to scenarios where the transfer of knowledge from the source to the target does not lead to any improvement, but rather causes a drop in the overall performance of the target task. There can be various reasons for negative transfer, such as cases when the source task is not sufficiently related to the target task, or if the transfer method could not leverage the relationship between the source and target tasks very well. Avoiding negative transfer is very important and requires careful investigation. In their work, Rosenstien and their co-authors present empirically how brute-force transfer degrades performance in target tasks when the source and target are too dissimilar. Bayesian approaches by Bakker and their co-authors, along with other techniques exploring clustering-based solutions to identify relatedness, are being researched to avoid negative transfers.
* Transfer Bounds: Quantifying the transfer in transfer learning is also very important, that affects the quality of the transfer and its viability. To gauge the amount for the transfer, Hassan Mahmud and their co-authors used Kolmogorov complexity to prove certain theoretical bounds to analyze transfer learning and measure relatedness between tasks. Eaton and their co-authors presented a novel graph-based approach to measure knowledge transfer. Detailed discussions of these techniques are outside the scope of this article. Readers are encouraged to explore more on these topics using the publications outlined in this section!

# Model to json
**inceptionV3**

In [ ]:
import json
from keras.models import load_model, model_from_json

# Load the Keras model from .h5 file
model = load_model('/kaggle/input/resnet50-model-pdd-v1/resnet50_pdd_model.h5')

# Extract model architecture to JSON
model_json = model.to_json()

# Save the JSON to a file
with open("resnet50.json", "w") as json_file:
    json_file.write(model_json)

# Load the model architecture from JSON
with open('resnet50.json', 'r') as json_file:
    loaded_model_json = json_file.read()

# Recreate the Keras model from the JSON
reconstructed_model = model_from_json(loaded_model_json)

# Check if the model is compiled. If yes, compile it again.
if hasattr(model, 'compile'):
    reconstructed_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Evaluate the reconstructed model
reconstructed_model.evaluate(x_test, y_test)  # Replace x_test, y_test with your test data


# model.h5 to pb and then .pb to json

In [8]:
import tensorflow as tf

# Load the Keras model from .h5 file
keras_model = tf.keras.models.load_model('/kaggle/input/densenet121-pdd-model-v1/densenet121pdd_model.h5')

# Convert the Keras model to a TensorFlow model
session = tf.compat.v1.keras.backend.get_session()

# Define the output node name
output_node_name = keras_model.output.op.name

# Ensure that there is an output node
if not output_node_name:
    print("No output node found in the model.")
    exit()

# Save the TensorFlow model using tf.saved_model
try:
    tf.saved_model.save(keras_model, './saved_model')
    print("Model successfully saved as a saved model.")
except Exception as e:
    print("Error occurred during saving the model:", str(e))
    exit()

# Load the saved model back
try:
    loaded_model = tf.saved_model.load('./saved_model')
except Exception as e:
    print("Error occurred during loading the saved model:", str(e))
    exit()

# Convert the loaded model to a TensorFlow graph
graph = tf.compat.v1.get_default_graph()

# Get the concrete function for the loaded model
concrete_function = loaded_model.signatures['serving_default']

# Freeze the TensorFlow graph
frozen_graph_def = tf.graph_util.convert_variables_to_constants(
    session,
    graph.as_graph_def(),
    [output_node_name]
)

# Save the TensorFlow model as .pb file
try:
    tf.io.write_graph(frozen_graph_def, '.', 'densenet121model.pb', as_text=False)
    print("Model successfully saved as 'densenet121model.pb'")
except Exception as e:
    print("Error occurred during saving the model:", str(e))


ValueError: Unknown layer: Functional

In [14]:
model = tf.keras.model.load_model('/kaggle/input/densenet121-pdd-model-v1/densenet121pdd_model.h5')
model.export_TF()

AttributeError: module 'tensorflow_core.keras' has no attribute 'model'

In [11]:
import tensorflow as tf
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras import layers

# Define the number of classes
class_count = 38 # define your number of classes here

# Define the model architecture
feats = DenseNet121(weights='imagenet', input_shape=(224, 224, 3), include_top=False)
feats.trainable = False

inp = tf.keras.Input(shape=(224, 224, 3))
x = feats(inp, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(class_count, activation='softmax')(x)

model = tf.keras.Model(inp, x)

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Save the model in .h5 format
model.save('your_model.h5')

# Convert the Keras model to a TensorFlow graph
session = tf.compat.v1.keras.backend.get_session()

# Define the output node name
output_node_name = model.output.op.name

# Ensure that there is an output node
if not output_node_name:
    print("No output node found in the model.")
    exit()

# Save the TensorFlow model using tf.saved_model
try:
    tf.saved_model.save(model, './saved_model')
    print("Model successfully saved as a saved model.")
except Exception as e:
    print("Error occurred during saving the model:", str(e))
    exit()

# Load the saved model back
try:
    loaded_model = tf.saved_model.load('./saved_model')
except Exception as e:
    print("Error occurred during loading the saved model:", str(e))
    exit()

# Convert the loaded model to a TensorFlow graph
graph = tf.compat.v1.get_default_graph()

# Get the concrete function for the loaded model
concrete_function = loaded_model.signatures['serving_default']

# Freeze the TensorFlow graph
frozen_graph_def = tf.compat.v1.graph_util.convert_variables_to_constants(
    session,
    graph.as_graph_def(),
    [output_node_name]
)

# Save the TensorFlow model as .pb file
try:
    tf.io.write_graph(frozen_graph_def, '.', 'your_model.pb', as_text=False)
    print("Model successfully saved as 'your_model.pb'")
except Exception as e:
    print("Error occurred during saving the model:", str(e))


29089792/29084464 [==============================] - 1s 0us/step
Model successfully saved as a saved model.


AssertionError: dense/Identity is not in graph

In [ ]:
import shutil

# Define the directory you want to download
directory_to_download = '/kaggle/working/saved_model'

# Define the name for the zip file
zip_file_name = 'densenet121model.zip'

# Create a zip archive of the directory
shutil.make_archive(zip_file_name.split('.')[0], 'zip', directory_to_download)

# Download the zip file
from IPython.display import FileLink
FileLink(zip_file_name)


In [ ]:
import os
import shutil

# Define the working directory path
working_dir = '/kaggle/working'

# Get a list of all files and directories in the working directory
contents = os.listdir(working_dir)

# Remove each file and directory in the working directory
for item in contents:
    item_path = os.path.join(working_dir, item)
    if os.path.isfile(item_path):
        os.remove(item_path)
    elif os.path.isdir(item_path):
        shutil.rmtree(item_path)

print("Contents of the working directory have been cleared.")


In [16]:
import tensorflow as tf

# Load the functional model from the saved model directory
loaded_model = tf.keras.models.load_model('/kaggle/input/densenet121-pdd-model-v1/densenet121pdd_model.h5')

# Check the summary of the loaded model
loaded_model.summary()


ValueError: Unknown layer: Functional